In [ ]:
import nltk
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import codecs
%matplotlib inline

Dataset from kaggle:

https://www.kaggle.com/datasets/venky73/spam-mails-dataset?select=spam_ham_dataset.csv

## Loading the dataset

In [ ]:
df = pd.read_csv("spam_ham_dataset.csv", index_col=0)

In [ ]:
df

In [ ]:
df.shape

In [ ]:
df["label_num"].value_counts()

In [ ]:
df['text'][12]

In [ ]:
def remove_line_break(text):
    # Removes all line breaks
    for sign in '\r\n':
        text = text.replace(sign, " ")
    return text

In [ ]:
df["text"]=df["text"].apply(remove_line_break)

In [ ]:
df['text'][12]

In [ ]:
df['lenght'] = df['text'].apply(lambda x: len(x))

In [ ]:
df['lenght'].hist(bins=100)

##Using TF

### Unigrams

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None,
                             stop_words = None, max_features = 15000)
data_text = vectorizer.fit_transform(df["text"])

In [ ]:
data_text.toarray()

In [ ]:
vectorizer.get_feature_names_out()

In [ ]:
vectorizer.get_feature_names_out()[1000:1010]

In [ ]:
vectorizer.get_feature_names_out()[10000:10010]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_text, df["label_num"], test_size=0.25)

In [ ]:
mnb = MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
clf=mnb.fit(X_train,y_train)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, mnb.predict(X_test))

### Unigrams without stop words

In [ ]:
vectorizer = CountVectorizer(analyzer = "word",tokenizer = None,preprocessor = None,
                             stop_words="english",max_features = 15000)
data_text = vectorizer.fit_transform(df["text"])

In [ ]:
data_text.shape

In [ ]:
vectorizer.get_feature_names_out()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_text, df["label_num"], test_size=0.25)
clf=mnb.fit(X_train,y_train)
accuracy_score(y_test, mnb.predict(X_test))

###Bigrams without stop words

In [ ]:
vectorizer = CountVectorizer(analyzer = "word",tokenizer = None,preprocessor = None,
                             stop_words="english", ngram_range=(2,2))
data_text = vectorizer.fit_transform(df["text"])

In [ ]:
data_text.shape

In [ ]:
vectorizer.get_feature_names_out()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_text, df["label_num"], test_size=0.25)
clf=mnb.fit(X_train,y_train)
accuracy_score(y_test, mnb.predict(X_test))

### N-grams (1)

In [ ]:
vectorizer = CountVectorizer(analyzer = "char_wb",tokenizer = None,
                             preprocessor = None,stop_words = None,max_features = 15000)
data_text = vectorizer.fit_transform(df["text"])

In [ ]:
data_text.shape

In [ ]:
vectorizer.get_feature_names_out()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_text, df["label_num"], test_size=0.25)
clf=mnb.fit(X_train,y_train)
accuracy_score(y_test, mnb.predict(X_test))

### N-grams (2)

In [ ]:
vectorizer = CountVectorizer(analyzer = "char_wb",tokenizer = None,preprocessor = None,
                             stop_words = None, ngram_range=(2,2), max_features = 15000)
data_text = vectorizer.fit_transform(df["text"])

In [ ]:
data_text.shape

In [ ]:
vectorizer.get_feature_names_out()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_text, df["label_num"], test_size=0.25)
clf=mnb.fit(X_train,y_train)
accuracy_score(y_test, mnb.predict(X_test))

### N-grams (3)

In [ ]:
vectorizer = CountVectorizer(analyzer = "char_wb",tokenizer = None,preprocessor = None,stop_words = None,
                             ngram_range=(3,3), max_features = 15000)
data_text = vectorizer.fit_transform(df["text"])

In [ ]:
data_text.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_text, df["label_num"], test_size=0.25)
clf=mnb.fit(X_train,y_train)
accuracy_score(y_test, mnb.predict(X_test))

###N-grams (4)

In [ ]:
vectorizer = CountVectorizer(analyzer = "char_wb",tokenizer = None,preprocessor = None,stop_words = None,
                             ngram_range=(4,4), max_features = 15000)
data_text = vectorizer.fit_transform(df["text"])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_text, df["label_num"], test_size=0.25)
clf=mnb.fit(X_train,y_train)
accuracy_score(y_test, mnb.predict(X_test))

###N-grams (5)

In [ ]:
vectorizer = CountVectorizer(analyzer = "char_wb",tokenizer = None,preprocessor = None,stop_words = None,
                             ngram_range=(5,5), max_features = 30000)
data_text = vectorizer.fit_transform(df["text"])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_text, df["label_num"], test_size=0.25)
clf=mnb.fit(X_train,y_train)
accuracy_score(y_test, mnb.predict(X_test))

## Using TF-IDF

###Unigrams

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(analyzer = "word", tokenizer = None,preprocessor = None,stop_words = None,max_features = 15000)
data_text = vectorizer.fit_transform(df["text"])

In [ ]:
vectorizer.get_feature_names_out()

In [ ]:
data_text.toarray()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_text, df["label_num"], test_size=0.25)
mnb = MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
clf=mnb.fit(X_train,y_train)
accuracy_score(y_test, mnb.predict(X_test))

###N-grams (5)

In [ ]:
vectorizer = TfidfVectorizer(analyzer = "char_wb",tokenizer = None,preprocessor = None,stop_words = None,
                             ngram_range=(5,5), max_features = 30000)
data_text = vectorizer.fit_transform(df["text"])

In [ ]:
data_text.toarray()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_text, df["label_num"], test_size=0.25)
clf=mnb.fit(X_train,y_train)
accuracy_score(y_test, mnb.predict(X_test))